# Reinforcement Learning Assignment  
**Points:** 120

---

## **Deep Q-Network (DQN) on Cartpole v1**  
**Homework 4**  
**Deep Learning Course**  
**Instructor:** Dr. Beigy  
**Term:** Fall 2024  

---

### **Student Information**  
- **Full Name:** Zahra Fallah MirMousavi
- **Student ID (SID):** 401207192

---


# Implementing Deep Q-Network (DQN) on CartPole v1  

---

In this notebook, we are going to implement **Deep Q-Network (DQN)** on the [CartPole v1 environment](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) using the **PyTorch** framework.  

We will:  

1. **Understand the Problem**:
   - Explore the CartPole v1 environment provided by OpenAI Gym.
   - Understand the state-space, action-space, and the reward structure of the environment.

2. **Implement the DQN Algorithm**:
   - Define the neural network architecture to approximate the Q-value function.
   - Implement the experience replay buffer to store and sample transitions.
   - Use the ε-greedy policy for balancing exploration and exploitation.
   - Implement the target network to stabilize training.

3. **Train the Agent**:
   - Define the training loop where the agent interacts with the environment.
   - Update the Q-network based on the Bellman equation.
   - Periodically update the target network.

4. **Evaluate Performance**:
   - Track and visualize the agent's performance during training.
   - Analyze the rewards and stability of the learned policy.

---

By the end of this notebook, you will gain a deeper understanding of how to implement DQN in practice and how to apply reinforcement learning techniques to solve control tasks.  


In [1]:
!pip install gym

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gym
from collections import deque
import random
import time

In [3]:
# Network Architecture (15 points)
# Define the DQN network architecture
class DQN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DQN, self).__init__()

        # TODO: Define the network architecture
        # Simple MLP with two hidden layers
        self.fc1 = nn.Linear(input_size, 24)
        self.fc2 = nn.Linear(24, 24)
        self.fc3 = nn.Linear(24, output_size)

    def forward(self, x):
        # TODO: Implement the forward pass
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# DQNAgent Initialization (15 points)
# Define the DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Hyperparameters
        # TODO: Adjust these hyperparameters if needed
        self.gamma = 0.95    # discount factor
        self.epsilon = 1.0   # initial exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.batch_size = 64

        # TODO: Initialize the replay memory
        self.memory = deque(maxlen=100000)

        # TODO: Create policy_net and target_net using DQN, and move them to self.device
        self.policy_net = DQN(state_size, action_size).to(self.device)
        self.target_net = DQN(state_size, action_size).to(self.device)

        # TODO: Copy policy_net weights to target_net
        self.update_target_network()

        # TODO: Create optimizer (e.g., Adam) for policy_net parameters
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)


    # Memory Handling (10 points)
    def remember(self, state, action, reward, next_state, done):
        # TODO: Store the experience (state, action, reward, next_state, done) in the replay memory
       self.memory.append((state, action, reward, next_state, done))

    # Action Selection (10 points)
    def act(self, state, evaluate=False):
        # TODO: Implement epsilon-greedy action selection:
        # If not in evaluate mode and random < epsilon, choose a random action
        # Otherwise, choose the best action from the policy network
        if (not evaluate) and (np.random.rand() <= self.epsilon):
            return np.random.randint(self.action_size)
        else:
            state_t = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.policy_net(state_t)
            return torch.argmax(q_values).item()


    # Experience Replay and Training Step (20 points)
    def replay(self):
        # Only train if we have enough samples in memory
        if len(self.memory) < self.batch_size:
            return

        # TODO: Sample a minibatch from replay memory
        minibatch = random.sample(self.memory, self.batch_size)

        # Prepare numpy arrays for states, actions, rewards, next_states, and done
        states = []
        actions = []
        rewards = []
        next_states = []
        dones = []

        for state, action, reward, next_state, done in minibatch:
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(done)

        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)

        # Compute predicted Q-values using policy_net (for the actions taken)
        q_values = self.policy_net(states).gather(1, actions).squeeze(1)

        # TODO: Compute target Q-values using target_net
        with torch.no_grad():
            # Next state max Q from target net
            max_next_q_values = self.target_net(next_states).max(1)[0]
            # target = reward + gamma * max_next_q_values * (1 - done)
            target_q_values = rewards + (self.gamma * max_next_q_values * (1 - dones))

        # Compute the loss (e.g., MSE or Huber)
        loss = nn.MSELoss()(q_values, target_q_values)

        # Perform a gradient update step
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Decrease epsilon if above epsilon_min
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


    # Target Network Updates (5 points)
    def update_target_network(self):
        # TODO: Update target_net parameters with policy_net parameters
        self.target_net.load_state_dict(self.policy_net.state_dict())

In [12]:
!pip install --upgrade gym pygame

  Using cached gym-0.26.2.tar.gz (721 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827626 sha256=14fa03077b298e2ef04bb98683da1b0ea21d4b6c54569868cc40f63503adbadd
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.0 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


In [13]:
# Training Loop (20 points)
def train_cartpole():
    # Create the environment
    env = gym.make("CartPole-v1", render_mode="human")
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    # TODO: Create an instance of DQNAgent
    agent = DQNAgent(state_size, action_size)


    episodes = 500
    target_update_frequency = 10
    scores = []

    for episode in range(episodes):
        # TODO: Reset the environment
        state = env.reset()

        score = 0
        done = False

        while not done:
            # TODO: Use agent.act to get an action
            action = agent.act(state)

            # TODO: Take the action in the environment, observe next_state, reward, and done
            next_state, reward, done, info = env.step(action)

            # TODO: Store the experience in the agent's memory (agent.remember)
            agent.remember(state, action, reward, next_state, done)

            # TODO: Call agent.replay() to update the policy_net
            agent.replay()

            # TODO: Update state and accumulate reward
            state = next_state
            score += reward

        # TODO: Update target network periodically
        if (episode + 1) % target_update_frequency == 0:
            agent.update_target_network()

        scores.append(score)
        mean_score = np.mean(scores[-100:])

        print(f"Episode: {episode + 1}, Score: {score}, Average Score: {mean_score:.2f}, Epsilon: {agent.epsilon:.3f}")  # TODO: Print agent.epsilon instead of 0.0

        # TODO: Early stopping condition if solved
        if mean_score >= 195.0 and episode >= 100:
            print(f"Solved in {episode + 1} episodes!")
            break

    env.close()
    # return agent, scores
    return agent, scores

In [7]:
# Visualization (5 points)
def visualize_agent(agent, num_episodes=5):
    """
    Visualize the trained agent in the environment.
    """
    # TODO: Implement evaluation loop without exploration (evaluate=True).
    # For each episode:
    # 1. Reset environment
    # 2. Render and step through with the agent's policy actions
    # 3. Print the score at the end of each episode
    env = gym.make('CartPole-v1')

    for e in range(num_episodes):
        state = env.reset()
        done = False
        score = 0

        # Temporarily set epsilon to 0 for pure exploitation
        original_epsilon = agent.epsilon
        agent.epsilon = 0.0

        while not done:
            env.render()
            action = agent.act(state, evaluate=True)
            next_state, reward, done, info = env.step(action)
            state = next_state
            score += reward

            # Slow down rendering if desired
            time.sleep(0.02)

        print(f"Episode {e+1} - Score: {score}")
        # Restore epsilon if needed
        agent.epsilon = original_epsilon

    env.close()


In [14]:
print("Training the agent...")
# TODO: Train the agent by calling train_cartpole and store the trained agent
# agent, scores = train_cartpole()
agent, scores = train_cartpole()

print("\nStarting visualization...")
# TODO: Visualize the trained agent by calling visualize_agent(agent)
visualize_agent(agent)

Training the agent...


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Episode: 1, Score: 19.0, Average Score: 19.00, Epsilon: 1.000
Episode: 2, Score: 19.0, Average Score: 19.00, Epsilon: 1.000
Episode: 3, Score: 18.0, Average Score: 18.67, Epsilon: 1.000
Episode: 4, Score: 18.0, Average Score: 18.50, Epsilon: 0.946
Episode: 5, Score: 11.0, Average Score: 17.00, Epsilon: 0.896
Episode: 6, Score: 21.0, Average Score: 17.67, Epsilon: 0.806
Episode: 7, Score: 17.0, Average Score: 17.57, Epsilon: 0.740
Episode: 8, Score: 10.0, Average Score: 16.62, Epsilon: 0.704
Episode: 9, Score: 21.0, Average Score: 17.11, Epsilon: 0.634
Episode: 10, Score: 16.0, Average Score: 17.00, Epsilon: 0.585
Episode: 11, Score: 22.0, Average Score: 17.45, Epsilon: 0.524
Episode: 12, Score: 24.0, Average Score: 18.00, Epsilon: 0.464
Episode: 13, Score: 11.0, Average Score: 17.46, Epsilon: 0.440
Episode: 14, Score: 18.0, Average Score: 17.50, Epsilon: 0.402
Episode: 15, Score: 11.0, Average Score: 17.07, Epsilon: 0.380
Episode: 16, Score: 12.0, Average Score: 16.75, Epsilon: 0.358
E

### ❓ **Question**

#### *(20 points)*  
> **Why is it important to maintain both a policy network and a target network in Deep Q-Network training, and how does the use of a target network help stabilize learning?**  


Maintaining *two* separate networks—often referred to as the **policy network** (or online network) and the **target network**—is a key technique in Deep Q-Network (DQN) training to improve *stability* and *reduce non-stationarity* of the learning signal. Here’s why:

1. **Non-stationary Targets**  
   - In standard Q-learning, the target for the Q-value update depends on the *same* Q function being updated (i.e., $Q(s, a)$ is used on both sides of the update).  
   - If we use a *single* network for both evaluating the current Q-values and calculating the target, then every time the policy network’s parameters are updated, it changes the target values for the next update. This creates a moving target problem— the model is always chasing a target that is itself shifting in unpredictable ways.

2. **Target Network as a Stable Reference**  
   - By having a **target network** (which is updated less frequently), you create a more *stable* or *fixed* reference for calculating target Q-values.  
   - Concretely, you do something like:  
     $$
       \text{target} = r + \gamma \max_{a'} Q_\text{target}(s', a')
     $$
     while the *policy network* (online network) is the one being trained to minimize:
     $$
       \Bigl(Q_\text{policy}(s, a) - \text{target}\Bigr)^2
     $$
   - Because $Q_\text{target}$ is not changing after *every* training step (it’s updated periodically or by a soft update rule), the learning process becomes more stable.

3. **Decoupling Updates**  
   - The target network helps *decouple* the action-value target computation from the parameter updates of the online network. This reduces the risk of feedback loops that can lead to wild oscillations in the Q-values.

4. **Practical Stabilization**  
   - Empirically, including a target network (updated, for instance, every fixed number of steps or via a slow “soft-update” mechanism) substantially reduces divergence and helps the network converge more reliably.

In short, the target network serves as a slowly moving or temporarily fixed function that the policy network can reliably learn from, thereby helping stabilize and improve learning performance in DQNs.
